In [6]:
import functools
from typing import Callable

import pandas as pd

from experiment_util import run_experiment, ModelBuilder
from edgedroid.models import timings
import numpy as np

window_size = 8
ttf_bins = 6
cleanup = timings.CleanupMode.TRUNCATE

num_steps = 100
num_reps = 40

def build_realistic_empirical(high_neuro: bool) -> tuple[str, timings.ExecutionTimeModel]:
    kernel = timings.LinearTTFWindowKernel(window_size=window_size, max_relative_weight=10)
    model = timings.EmpiricalETM(kernel=kernel, neuroticism=1.0 if high_neuro else 0.0, ttf_levels=ttf_bins, cleanup=cleanup)
    name = f"Realistic ({'high' if high_neuro else 'low'} neuro)"
    return name, model

def build_realistic_empirical_mean(high_neuro: bool) -> tuple[str, timings.ExecutionTimeModel]:
    kernel = timings.LinearTTFWindowKernel(window_size=window_size, max_relative_weight=10)
    model = timings.EmpiricalAggregateETM(aggregate_fn=np.mean, kernel=kernel, neuroticism=1.0 if high_neuro else 0.0, ttf_levels=ttf_bins, cleanup=cleanup)
    name = f"Realistic Mean ({'high' if high_neuro else 'low'} neuro)"
    return name, model

def build_realistic_fitted(high_neuro: bool) -> tuple[str, timings.ExecutionTimeModel]:
    kernel = timings.LinearTTFWindowKernel(window_size=window_size, max_relative_weight=10)
    model = timings.FittedETM(kernel=kernel, neuroticism=1.0 if high_neuro else 0.0, ttf_levels=ttf_bins, cleanup=cleanup)
    name = f"Realistic Fitted ({'high' if high_neuro else 'low'} neuro)"
    return name, model

def high_neuro(builder_fn: Callable[[bool], tuple[str, timings.ExecutionTimeModel]]) -> ModelBuilder:
    return functools.partial(builder_fn, high_neuro=True)

def low_neuro(builder_fn: Callable[[bool], tuple[str, timings.ExecutionTimeModel]]) -> ModelBuilder:
    return functools.partial(builder_fn, high_neuro=True)

def build_legacy() -> tuple[str, timings.ExecutionTimeModel]:
    kernel = timings.LinearTTFWindowKernel(window_size=window_size, max_relative_weight=10)
    model = timings.LegacyETM(kernel, ttf_levels=ttf_bins)
    name = "Legacy"
    return name, model

def build_baseline() -> tuple[str, timings.ExecutionTimeModel]:
    return "Baseline", timings.FirstOrderETM()

def low_ttf_seq_builder() -> list[float]:
    return [0.1] * num_steps

def mid_ttf_seq_builder() -> list[float]:
    return [1.6] * num_steps

def high_ttf_seq_builder() -> list[float]:
    return [3.0] * num_steps

low_ttf_results = run_experiment(
    model_builders=[
        high_neuro(build_realistic_empirical),
        low_neuro(build_realistic_empirical),
        high_neuro(build_realistic_empirical_mean),
        low_neuro(build_realistic_empirical_mean),
        high_neuro(build_realistic_fitted),
        low_neuro(build_realistic_fitted),
        build_legacy,
        build_baseline,
    ],
    ttf_seq_builder=low_ttf_seq_builder,
    reps_per_setup=num_reps,
)
low_ttf_results["ttf_level"] = "low"

mid_ttf_results = run_experiment(
    model_builders=[
        high_neuro(build_realistic_empirical),
        low_neuro(build_realistic_empirical),
        high_neuro(build_realistic_empirical_mean),
        low_neuro(build_realistic_empirical_mean),
        high_neuro(build_realistic_fitted),
        low_neuro(build_realistic_fitted),
        build_legacy,
        build_baseline,
    ],
    ttf_seq_builder=mid_ttf_seq_builder,
    reps_per_setup=num_reps,
)
mid_ttf_results["ttf_level"] = "mid"

high_ttf_results = run_experiment(
    model_builders=[
        high_neuro(build_realistic_empirical),
        low_neuro(build_realistic_empirical),
        high_neuro(build_realistic_empirical_mean),
        low_neuro(build_realistic_empirical_mean),
        high_neuro(build_realistic_fitted),
        low_neuro(build_realistic_fitted),
        build_legacy,
        build_baseline,
    ],
    ttf_seq_builder=high_ttf_seq_builder,
    reps_per_setup=num_reps,
)
high_ttf_results["ttf_level"] = "high"

results = pd.concat((low_ttf_results, mid_ttf_results, high_ttf_results))
results["run_id"] = results["run_id"].astype("category")
results["ttf_level"] = results["ttf_level"].astype("category")
results.to_parquet("./fixed_ttf_results.parquet")
results

,run_id,model_name,prev_ttf,step_index,exec_time,step_time,task_duration,ttf_level
0,c5372fd77dcf40ef9feba61a03ca42f9,Baseline,0.0,0,5.170087,5.220087,5.220087,low
1,c5372fd77dcf40ef9feba61a03ca42f9,Baseline,0.1,1,6.124708,6.174708,11.394795,low
2,c5372fd77dcf40ef9feba61a03ca42f9,Baseline,0.1,2,8.156299,8.206299,19.601094,low
3,c5372fd77dcf40ef9feba61a03ca42f9,Baseline,0.1,3,6.243062,6.293062,25.894156,low
4,c5372fd77dcf40ef9feba61a03ca42f9,Baseline,0.1,4,1.841317,1.891317,27.785473,low
...,...,...,...,...,...,...,...,...
95,46d86f9cb94f49bfa0e8a722e140cc36,Realistic Fitted (high neuro),3.0,95,3.165549,4.665549,697.534427,high
96,46d86f9cb94f49bfa0e8a722e140cc36,Realistic Fitted (high neuro),3.0,96,5.973634,7.473634,705.008061,high
97,46d86f9cb94f49bfa0e8a722e140cc36,Realistic Fitted (high neuro),3.0,97,4.659607,6.159607,711.167667,high
98,46d86f9cb94f49bfa0e8a722e140cc36,Realistic Fitted (high neuro),3.0,98,5.843266,7.343266,718.510933,high
